# Python Lectures (2)

Last time we did a brief introduction. We were writing a small application that calculates currency exchange. This time we are going to extend the application by pulling the exchange rates from the web.  For that I am going to need two libraries:

In [6]:
import urllib
import xmltodict

The first library `urllib` is for pulling content from the web, while the second `xmltodict` for parsing an XML stream into a python dictionary (map).

In [7]:
raw = xmltodict.parse(urllib.request.urlopen("http://www.ecb.europa.eu/stats/eurofxref/eurofxref-daily.xml"))

[The European Central Bank](http://www.ecb.europa.eu/) keeps daily data on exchange rate at the URL you see above.  The data is kept in [XML](http://en.wikipedia.org/wiki/XML) format.  When we call the function `urllib.urlopen` on the URL, we get a python stream: something like a file.  Then the function `xmltodict.parse` parses the content of the stream into a python map.  The contents of the map complicated:

In [8]:
raw

OrderedDict([('gesmes:Envelope',
              OrderedDict([('@xmlns:gesmes',
                            'http://www.gesmes.org/xml/2002-08-01'),
                           ('@xmlns',
                            'http://www.ecb.int/vocabulary/2002-08-01/eurofxref'),
                           ('gesmes:subject', 'Reference rates'),
                           ('gesmes:Sender',
                            OrderedDict([('gesmes:name',
                                          'European Central Bank')])),
                           ('Cube',
                            OrderedDict([('Cube',
                                          OrderedDict([('@time', '2017-09-22'),
                                                       ('Cube',
                                                        [OrderedDict([('@currency',
                                                                       'USD'),
                                                                      ('@rate',
                    

Everything in python is an object, and objects have admits interfaces. Maps have two fundamental interfaces: keys and values.

In [9]:
Rates = {"US": 1.0, "EUR": 0.8, "TRY": 2.24, "BITCOIN": 0.0026}

print(Rates.keys())
print(Rates.values())

dict_keys(['TRY', 'EUR', 'US', 'BITCOIN'])
dict_values([2.24, 0.8, 1.0, 0.0026])


So, let us see what the keys are in the raw data we pulled of from the European Central Bank:

In [10]:
raw.keys()

odict_keys(['gesmes:Envelope'])

This is funny. There is only one key. If you do data analysis long enough, you'll see that XML, or JSON, data you pulled is structured as smartly (or stupidly) as the people who design them.  Let us move in recursively:

In [11]:
raw[u'gesmes:Envelope']

OrderedDict([('@xmlns:gesmes', 'http://www.gesmes.org/xml/2002-08-01'),
             ('@xmlns', 'http://www.ecb.int/vocabulary/2002-08-01/eurofxref'),
             ('gesmes:subject', 'Reference rates'),
             ('gesmes:Sender',
              OrderedDict([('gesmes:name', 'European Central Bank')])),
             ('Cube',
              OrderedDict([('Cube',
                            OrderedDict([('@time', '2017-09-22'),
                                         ('Cube',
                                          [OrderedDict([('@currency', 'USD'),
                                                        ('@rate', '1.1961')]),
                                           OrderedDict([('@currency', 'JPY'),
                                                        ('@rate', '134.01')]),
                                           OrderedDict([('@currency', 'BGN'),
                                                        ('@rate', '1.9558')]),
                                           Ordere

Another map/dictionary! Hmm... The keys?

In [12]:
raw[u'gesmes:Envelope'].keys()

odict_keys(['@xmlns:gesmes', '@xmlns', 'gesmes:subject', 'gesmes:Sender', 'Cube'])

In [14]:
raw[u'gesmes:Envelope'][u'Cube'][u'Cube'][u'Cube']

[OrderedDict([('@currency', 'USD'), ('@rate', '1.1961')]),
 OrderedDict([('@currency', 'JPY'), ('@rate', '134.01')]),
 OrderedDict([('@currency', 'BGN'), ('@rate', '1.9558')]),
 OrderedDict([('@currency', 'CZK'), ('@rate', '26.046')]),
 OrderedDict([('@currency', 'DKK'), ('@rate', '7.4407')]),
 OrderedDict([('@currency', 'GBP'), ('@rate', '0.88155')]),
 OrderedDict([('@currency', 'HUF'), ('@rate', '309.73')]),
 OrderedDict([('@currency', 'PLN'), ('@rate', '4.2672')]),
 OrderedDict([('@currency', 'RON'), ('@rate', '4.5967')]),
 OrderedDict([('@currency', 'SEK'), ('@rate', '9.5358')]),
 OrderedDict([('@currency', 'CHF'), ('@rate', '1.1588')]),
 OrderedDict([('@currency', 'NOK'), ('@rate', '9.3193')]),
 OrderedDict([('@currency', 'HRK'), ('@rate', '7.4843')]),
 OrderedDict([('@currency', 'RUB'), ('@rate', '68.8885')]),
 OrderedDict([('@currency', 'TRY'), ('@rate', '4.1804')]),
 OrderedDict([('@currency', 'AUD'), ('@rate', '1.5011')]),
 OrderedDict([('@currency', 'BRL'), ('@rate', '3.7459'

This is really stupid. The data is an array of small dictionaries each of size 1 containing the name of the currency and the rate.  We need to put the data in the correct format.

In [15]:
stupid = raw[u'gesmes:Envelope'][u'Cube'][u'Cube'][u'Cube']

Rates = { x['@currency']: float(x[u'@rate']) for x in stupid }

Above, I used a construction called *comprehension*. Let me explain: assume you want a range of integers for some reason.  In python you call

In [17]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [18]:
list(range(2,12))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [19]:
list(range(24,3,-2))

[24, 22, 20, 18, 16, 14, 12, 10, 8, 6, 4]

But what if I want the squares of the even numbers between 2 and 26 stored in an array?

In [20]:
[ x*x for x in range(2,26,2) ]

[4, 16, 36, 64, 100, 144, 196, 256, 324, 400, 484, 576]

How about stored in a set?

In [21]:
{ x*x for x in range(2,26,2) }

{4, 16, 36, 64, 100, 144, 196, 256, 324, 400, 484, 576}

How about a dictionary?

In [23]:
{ "Index-"+str(i): i*i for i in range(2,26,2) }

{'Index-10': 100,
 'Index-12': 144,
 'Index-14': 196,
 'Index-16': 256,
 'Index-18': 324,
 'Index-2': 4,
 'Index-20': 400,
 'Index-22': 484,
 'Index-24': 576,
 'Index-4': 16,
 'Index-6': 36,
 'Index-8': 64}

Let us check the contents of `stupid`:

In [24]:
stupid[1]

OrderedDict([('@currency', 'JPY'), ('@rate', '134.01')])

In [25]:
stupid[1][u'@currency']

'JPY'

In [26]:
stupid[1][u'@rate']

'134.01'

In [27]:
Rates = { x['@currency']: float(x[u'@rate']) for x in stupid }

In [28]:
Rates

{'AUD': 1.5011,
 'BGN': 1.9558,
 'BRL': 3.7459,
 'CAD': 1.4675,
 'CHF': 1.1588,
 'CNY': 7.8805,
 'CZK': 26.046,
 'DKK': 7.4407,
 'GBP': 0.88155,
 'HKD': 9.3414,
 'HRK': 7.4843,
 'HUF': 309.73,
 'IDR': 15908.13,
 'ILS': 4.1755,
 'INR': 77.501,
 'JPY': 134.01,
 'KRW': 1352.78,
 'MXN': 21.2905,
 'MYR': 5.0117,
 'NOK': 9.3193,
 'NZD': 1.6336,
 'PHP': 60.532,
 'PLN': 4.2672,
 'RON': 4.5967,
 'RUB': 68.8885,
 'SEK': 9.5358,
 'SGD': 1.6093,
 'THB': 39.579,
 'TRY': 4.1804,
 'USD': 1.1961,
 'ZAR': 15.8359}

Notice that "EUR" is missing since this is a data from European Central Banks, the rates are all relative to "EUR". We add that extra:

In [29]:
Rates.update({u'EUR': 1.00})

In [30]:
Rates

{'AUD': 1.5011,
 'BGN': 1.9558,
 'BRL': 3.7459,
 'CAD': 1.4675,
 'CHF': 1.1588,
 'CNY': 7.8805,
 'CZK': 26.046,
 'DKK': 7.4407,
 'EUR': 1.0,
 'GBP': 0.88155,
 'HKD': 9.3414,
 'HRK': 7.4843,
 'HUF': 309.73,
 'IDR': 15908.13,
 'ILS': 4.1755,
 'INR': 77.501,
 'JPY': 134.01,
 'KRW': 1352.78,
 'MXN': 21.2905,
 'MYR': 5.0117,
 'NOK': 9.3193,
 'NZD': 1.6336,
 'PHP': 60.532,
 'PLN': 4.2672,
 'RON': 4.5967,
 'RUB': 68.8885,
 'SEK': 9.5358,
 'SGD': 1.6093,
 'THB': 39.579,
 'TRY': 4.1804,
 'USD': 1.1961,
 'ZAR': 15.8359}

Now, let us recall our program:

In [31]:
def Exchange(amount, source, target):
    return(amount * Rates[target]/Rates[source])

print(Exchange(100.0, "TRY", u'EUR'))

23.921155870251653


Let me write a new function that updates the exchange rate dictionary:

In [32]:
def UpdateRates():
    raw = xmltodict.parse(urllib.urlopen("http://www.ecb.europa.eu/stats/eurofxref/eurofxref-daily.xml"))
    core = raw[u'gesmes:Envelope'][u'Cube'][u'Cube'][u'Cube']
    Rates = { x['@currency']: float(x[u'@rate']) for x in core }

## A Finished Version

I prefer to interact with my program on the command line.  So, I will everything together as follows:

    ## Back End

    import urllib
    import xmltodict

    URL = "http://www.ecb.europa.eu/stats/eurofxref/eurofxref-daily.xml"

    def DownloadRates(localFile):
      urllib.urlretrieve(URL, filename=localFile)

    def GetRates(localFile):
      raw = xmltodict.parse(open(localFile))
      core = raw[u'gesmes:Envelope'][u'Cube'][u'Cube'][u'Cube']
      Rates = { x['@currency']: float(x[u'@rate']) for x in core }
      Rates.update({u'EUR': 1.00})
      return(Rates)

    def Exchange(Rates,amount,source,target):
      return(amount * Rates[target]/Rates[source])


    ## Front End

    import sys

    FILE = "/home/kaygun/local/lib/rates.xml"

    if(sys.argv[1] == "refresh"):
      DownloadRates(FILE)
    elif(sys.argv[1] == "list"):
      Rates = GetRates(FILE)
      Currencies = Rates.keys()
      Currencies.sort()
      for x in Currencies:
        print x, Rates[x]
    else:
      Rates = GetRates(FILE)
      amount = float(sys.argv[1])
      source = sys.argv[2]
      target = sys.argv[3]
      print str(amount), source, "is", "{0:.2f}".format(Exchange(Rates,amount,source,target)), target
